# Classifying classes in the latent space
Domain invariant classifier in the latent space

In [37]:
import scripts.init_gpu as init_gpu
import scripts.init_dataset as init_dataset
import pandas as pd
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '1'
init_gpu.initialize_gpus()

locations = ['LOC1', 'LOC2']

print("Loading Dataset...")
# load the dataset
df = pd.read_csv(
    f"../dataset/processed/{locations[0]}-{locations[1]}-scaled-balanced.csv")

length = len(df.columns) - 2  # subtract the two label columns

# get train-test set
train_df, test_df, train_web_sam1ples, test_web_samples = init_dataset.get_sample(
    df, locations, range(1500), 1200)

train_df.head()

Num GPUs Available:  2
Loading Dataset...
Training Websites: [1309, 228, 51, 563, 501, 457, 285, 209, 1385, 1116, 178, 1209, 864, 65, 61, 191, 447, 476, 1034, 1232, 54, 1149, 407, 1466, 1330, 1436, 1490, 859, 451, 919, 1206, 569, 13, 326, 1429, 865, 696, 1468, 318, 440, 689, 1492, 189, 778, 198, 735, 704, 1236, 541, 88, 940, 1098, 255, 775, 161, 1130, 600, 1287, 1266, 740, 1182, 393, 142, 93, 1354, 466, 592, 163, 1482, 206, 1456, 1462, 928, 1301, 747, 333, 758, 727, 429, 1372, 546, 1399, 1327, 146, 1247, 1300, 350, 1093, 1495, 334, 946, 777, 552, 1310, 1140, 449, 1402, 664, 114, 469, 1486, 646, 821, 548, 135, 432, 1161, 644, 435, 1342, 1022, 810, 1316, 939, 292, 542, 1493, 505, 1478, 1103, 538, 1197, 877, 1195, 817, 741, 1404, 283, 1043, 1010, 186, 96, 224, 313, 1285, 327, 1487, 1221, 130, 788, 781, 1220, 958, 1083, 514, 1133, 23, 234, 1099, 1419, 1312, 1463, 1498, 601, 890, 323, 929, 6, 539, 1025, 365, 1039, 217, 1280, 611, 1308, 1338, 1415, 1477, 1366, 765, 330, 1104, 1086, 1, 1226, 

/home/asil0892/doh_traffic_analysis/code/scripts/init_dataset.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.sort_values(by=["Location"], inplace=True)


,Location,Website,0,1,2,3,4,5,6,7,...,116,117,118,119,120,121,122,123,124,125
0,LOC1,0,-1.086861,0.690199,0.647933,0.201517,0.207139,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604
1,LOC1,1005,1.095547,-1.585943,0.647933,0.343653,0.207139,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604
2,LOC1,1005,1.095547,0.780047,-1.972790,0.343653,0.207139,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604
3,LOC1,1005,-1.086861,0.690199,0.647933,0.343653,0.207139,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604
4,LOC1,1005,1.095547,0.780047,0.183501,-0.828965,-2.083179,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604


In [38]:
from scripts.train_vae import VAE, Sampling, ConvVAE_BatchNorm
import tensorflow as tf
import numpy as np

# web_model = tf.keras.models.load_model(f"../models/website/{locations[0]}-{locations[1]}-baseGRU-epochs300-train_samples1200-triplet_samples5.keras")

# location = 'LOC1'
vae_model = tf.keras.models.load_model("../models/vae/ci_vae/ConvBased/domain_and_class/LOC1-LOC2-e880-mse1-kl0.01-cl1.0-ConvBatchNorm-ldim96-hdim128.keras", custom_objects={'ConvVAE_BatchNorm': ConvVAE_BatchNorm, 'Sampling': Sampling})

def get_z_embeddings(data, vae_model=vae_model):
    embeddings = []
    chunk_size = 200
    for i in range(0, len(data), chunk_size):
        chunk = data[i:i+chunk_size]
        _, _, transformed_chunk = vae_model.encode(chunk)
        embeddings.append(transformed_chunk)

    return np.vstack(embeddings)

/home/asil0892/doh_traffic_analysis/.venv/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [39]:
from sklearn.preprocessing import LabelEncoder

# get the training data
# from train set
x_train_traces = train_df.iloc[:, 2:].to_numpy().astype(np.float32)

# get the classes from the available data in the LOC1 but missing in LO2 
x_available_classes_from_source = test_df[test_df['Location']
                                            == 'LOC1'].iloc[:, 2:].to_numpy().astype(np.float32)
# combine both: duplicate the data for missing classes 
x_train_traces = np.vstack((x_train_traces, x_available_classes_from_source, x_available_classes_from_source))

# get latent embeddings
x_train = get_z_embeddings(x_train_traces, vae_model)

# get classes for class informed latent space
le = LabelEncoder()
y_train = train_df.Website
y_available_classes = test_df[test_df['Location']
                                == 'LOC1'].Website.to_numpy()
y_train = le.fit_transform(np.hstack((y_train, y_available_classes, y_available_classes)))

# Get domain labels (0 for LOC1, 1 for LOC2)
d_train = (train_df.Location == 'LOC2').astype(int)
d_available_classes = np.zeros(len(y_available_classes))  # All from LOC1, so 0
d_train = np.hstack((d_train, d_available_classes, d_available_classes))

In [40]:
# Test data
x_test_traces = test_df[test_df['Location'] == 'LOC2'].iloc[:, 2:].to_numpy().astype(np.float32)
x_test = get_z_embeddings(x_test_traces, vae_model)

y_test = le.transform(test_df[test_df['Location'] == 'LOC2'].Website.to_numpy())
d_test = np.ones(len(y_test))


In [41]:
# from xgboost import XGBClassifier
# from scripts.classification import evaluate_classification_model

# clf = XGBClassifier()
# evaluate_classification_model(x_train, y_train, x_test, y_test, clf)

# Domain Adapting Classifier Network

In [42]:
from tensorflow import keras


class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self, lambda_value=1.0, **kwargs):
        super(GradientReversalLayer, self).__init__(**kwargs)
        self.lambda_value = lambda_value

    def call(self, x):
        @tf.custom_gradient
        def grad_reverse(x):
            def custom_grad(dy):
                return -self.lambda_value * dy
            return x, custom_grad
        return grad_reverse(x)
    
def classifier_network(input_dim, num_classes, output_name):
    return keras.Sequential([
        # First dense layer
        keras.layers.Dense(
            1024,
            activation='relu',
            input_shape=(input_dim,),
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            kernel_regularizer=keras.regularizers.l2(1e-4)
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        
        # Second dense layer
        keras.layers.Dense(
            512,
            activation='relu',
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            kernel_regularizer=keras.regularizers.l2(1e-4)
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        
        # Output layer
        keras.layers.Dense(
            num_classes,
            activation=None,  # No activation for logits
            kernel_initializer='glorot_uniform',  # Better for linear activation
            bias_initializer='zeros',
            kernel_regularizer=keras.regularizers.l2(1e-4),
            name=output_name
        )
    ])
def cnn_classifier_network(input_shape, num_classes, output_name):
    return keras.Sequential([
        # Reshape the input to add the channel dimension
        keras.layers.InputLayer(input_shape=(input_shape,)),
        keras.layers.Reshape((-1, 1)),  # Reshape to (input_shape, 1) for Conv1D
        keras.layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        keras.layers.MaxPooling1D(pool_size=2),
        keras.layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'),
        keras.layers.MaxPooling1D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(num_classes, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(num_classes, activation=None, name=output_name)
    ])  
    
def build_domain_invariant_classifier(latent_dim, feature_extractor, class_classifier, domain_classifier, lambda_value=1.0):
    # Input
    inputs = keras.layers.Input(shape=(latent_dim, 1), name="input_layer")
    
    # Extract features
    features = feature_extractor(inputs)
    
    # Class predictor
    class_output = class_classifier(features)
    class_output = tf.keras.layers.Lambda(lambda x: tf.identity(x), name="class_output")(class_output)  # Explicitly name the output
    
    # Gradient reversal and domain classifier
    reversed_features = GradientReversalLayer(lambda_value=lambda_value)(features)
    domain_output = domain_classifier(reversed_features)
    domain_output = tf.keras.layers.Lambda(lambda x: tf.identity(x), name="domain_output")(domain_output)  # Explicitly name the output
    
    # Build the model
    model = keras.Model(inputs, [class_output, domain_output], name="domain_invariant_classifier")
    return model



# Define input shape and parameters
latent_dim = 96  # Example input feature size
num_classes = 1500    # Number of classes
feature_dim = 2048
num_domains = 2      # Number of domains
lambda_ = 0.1        # Gradient reversal strength

domain_classifier = classifier_network(feature_dim, num_domains, "domain_output")
class_classifier = keras.Sequential([
        # Input layer with normalization
        keras.layers.InputLayer(input_shape=(feature_dim,)),
        keras.layers.BatchNormalization(),
        
        # First dense block - expanding features
        keras.layers.Dense(
            2048,
            activation='relu',
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            kernel_regularizer=keras.regularizers.l2(1e-4)
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        
        # Second dense block - maintaining width
        keras.layers.Dense(
            2048,
            activation='relu',
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            kernel_regularizer=keras.regularizers.l2(1e-4)
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        
        # Third dense block - still wide
        keras.layers.Dense(
            1800,
            activation='relu',
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            kernel_regularizer=keras.regularizers.l2(1e-4)
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        
        # Output layer
        keras.layers.Dense(
            num_classes,
            activation=None,
            kernel_initializer='glorot_uniform',
            bias_initializer='zeros',
            kernel_regularizer=keras.regularizers.l2(1e-4),
            name='class_output'
        )
    ])
feature_extractor = keras.Sequential([
    keras.layers.InputLayer(input_shape=(latent_dim, 1)),
    
    # First conv block
    keras.layers.Conv1D(
        filters=128,
        kernel_size=5,
        activation='relu',
        padding='same',
        kernel_initializer='he_normal',
        bias_initializer='zeros',
        kernel_regularizer=keras.regularizers.l2(1e-4)
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=2),  # 48
    
    # Second conv block
    keras.layers.Conv1D(
        filters=256,
        kernel_size=3,
        activation='relu',
        padding='same',
        kernel_initializer='he_normal',
        bias_initializer='zeros',
        kernel_regularizer=keras.regularizers.l2(1e-4)
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=2),  # 24
    
    # Third conv block
    keras.layers.Conv1D(
        filters=256,
        kernel_size=3,
        activation='relu',
        padding='same',
        kernel_initializer='he_normal',
        bias_initializer='zeros',
        kernel_regularizer=keras.regularizers.l2(1e-4)
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=2),  # 12
    
    # Dense layers
    keras.layers.Flatten(),  # 12 * 256
    keras.layers.Dense(
        feature_dim,
        activation='relu',
        kernel_initializer='he_normal',
        bias_initializer='zeros',
        kernel_regularizer=keras.regularizers.l2(1e-4)
    ),
    keras.layers.BatchNormalization()
])

/home/asil0892/doh_traffic_analysis/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/asil0892/doh_traffic_analysis/.venv/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
lambda_ = 1.0       # Gradient reversal strength

model = build_domain_invariant_classifier(latent_dim, feature_extractor, class_classifier, domain_classifier, lambda_)
print(model.output_names)

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    loss={
        "class_output": keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        "domain_output": keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    },
    metrics={
        "class_output": "accuracy",
        "domain_output": "accuracy"
    },
    loss_weights={"class_output": 1.0, "domain_output": 1.0}
)    

model.fit(x_train, {"class_output": y_train, "domain_output": d_train}, batch_size=128, epochs=100, shuffle=True)
feature_extractor.save("../models/classification/latent_web/feature_extractor.keras")
class_classifier.save("../models/classification/latent_web/class_classifier.keras")
domain_classifier.save("../models/classification/latent_web/domain_classifier.keras")

ListWrapper(['class_output', 'domain_output'])
Epoch 1/100


I0000 00:00:1732258211.770156   20255 service.cc:148] XLA service 0x7f3d00015940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732258211.770170   20255 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
I0000 00:00:1732258211.770172   20255 service.cc:156]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-11-22 17:50:11.903363: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-22 17:50:13.181372: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.5.82. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA f

  55/4688 ━━━━━━━━━━━━━━━━━━━━ 28:46 373ms/step - class_output_accuracy: 3.6293e-04 - class_output_loss: 7.9708 - domain_output_accuracy: 0.4692 - domain_output_loss: 12.0200 - loss: 22.1902

2024-11-22 17:50:33.918785: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4742', 104 bytes spill stores, 104 bytes spill loads

2024-11-22 17:50:34.088704: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4103', 116 bytes spill stores, 120 bytes spill loads

2024-11-22 17:50:34.142014: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4103', 132 bytes spill stores, 132 bytes spill loads

2024-11-22 17:50:34.160567: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4103', 80 bytes spill stores, 80 bytes spill loads

2024-11-22 17:50:34.161427: I external/local_xla/xla/s

4688/4688 ━━━━━━━━━━━━━━━━━━━━ 43s 8ms/step - class_output_accuracy: 0.0066 - class_output_loss: 7.4945 - domain_output_accuracy: 0.5050 - domain_output_loss: 10.4389 - loss: 20.1167
Epoch 2/100
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 44s 9ms/step - class_output_accuracy: 0.1766 - class_output_loss: 5.1511 - domain_output_accuracy: 0.5100 - domain_output_loss: 8.7085 - loss: 16.0327
Epoch 3/100
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 44s 9ms/step - class_output_accuracy: 0.4793 - class_output_loss: 3.0615 - domain_output_accuracy: 0.5041 - domain_output_loss: 8.1886 - loss: 13.4159
Epoch 4/100
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 44s 9ms/step - class_output_accuracy: 0.6393 - class_output_loss: 1.9857 - domain_output_accuracy: 0.5149 - domain_output_loss: 7.6374 - loss: 11.7788
Epoch 5/100
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 42s 9ms/step - class_output_accuracy: 0.7098 - class_output_loss: 1.5032 - domain_output_accuracy: 0.5360 - domain_output_loss: 7.2248 - loss: 10.8683
Epoch 6/100
4688/4688 ━━━━━━━━━━━━━━━━━━━

In [43]:
class_classifier = tf.keras.models.load_model("../models/classification/latent_web/class_classifier.keras")
feature_extractor = tf.keras.models.load_model("../models/classification/latent_web/feature_extractor.keras")

In [44]:
# Get logits from the model
logits = class_classifier(feature_extractor(x_test))

# Convert logits to predictions
predictions = tf.argmax(logits, axis=1)

# Calculate accuracy
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(y_test, predictions)
acc_value = accuracy.result().numpy()


print("Accuracy", acc_value)

Accuracy 0.64783335


# see whether sending the test data to the other side of the location hyperplane helps

In [51]:
w_norm = [ 0.06975281,  0.04118532, -0.14468005, -0.05042024,  0.01549432,
       -0.02282231,  0.12847888, -0.01408303,  0.02656271,  0.0355668 ,
       -0.05648927, -0.14249759, -0.50905263,  0.01057976, -0.12150489,
        0.01641565, -0.06888216, -0.03849784, -0.00694003, -0.00515264,
        0.01254265, -0.36203945,  0.01098717, -0.21074273,  0.05474511,
        0.00883125,  0.0365044 , -0.04460574,  0.04653245,  0.02774703,
        0.05174151, -0.01852731, -0.00529856, -0.16091824,  0.18659344,
       -0.04393233,  0.04611056,  0.04562994,  0.02683887, -0.02065428,
       -0.00894934, -0.0686677 ,  0.04277077, -0.01343503, -0.0336921 ,
        0.01214135, -0.05644768, -0.01702002, -0.04333098,  0.07054286,
        0.0393395 , -0.12504448,  0.04985355, -0.06638476, -0.02668105,
       -0.0404169 ,  0.0033951 ,  0.10164927,  0.02533227, -0.00456006,
       -0.03698264,  0.01445536,  0.07971338, -0.05770196,  0.01214109,
       -0.01428591, -0.01922118, -0.05471335,  0.06706536,  0.09856064,
       -0.02785856, -0.06674519,  0.02121052, -0.02505254, -0.00223312,
       -0.0595909 , -0.07537825, -0.11761025, -0.02598793, -0.05158163,
       -0.024724  ,  0.02093302,  0.18926618, -0.00074534,  0.01370208,
       -0.02600524, -0.03602418,  0.06587068,  0.41776927, -0.10614276,
       -0.03901233,  0.12387   ,  0.07350189, -0.00673966,  0.1447743 ,
        0.0108655 ]

bias = [1.0903536]

def jump_to_other_side(z, w, b, alpha=0.1, mirrored=True):
    """
    Jump directly to the other side of the hyperplane.
    
    Parameters:
    - z: The current latent vector (numpy array).
    - w: The normal vector of the hyperplane.
    - b: The bias term of the hyperplane.
    - alpha: Additional step size to cross to the other side.
    
    Returns:
    - z_new: The updated latent vector on the other side.
    """
    # Compute the current value of the decision function
    decision_value = np.dot(w, z) + b
    
    # Compute the displacement to the hyperplane
    delta_z = -decision_value / np.dot(w, w) * w
    
    if mirrored:
        # mirror image of the point
        z_new = z + 1 * delta_z
    else: 
        # Add an additional step to cross the hyperplane
        z_new = z + delta_z + alpha * w

    return z_new

x_test_mirror = np.zeros(x_test.shape)
for i in range(len(x_test)):
    x_test_mirror[i] = jump_to_other_side(x_test[i], w_norm, bias, mirrored=True, alpha = 1.0) 
    
    
# Get logits from the model
logits = class_classifier(feature_extractor(x_test_mirror))

# Convert logits to predictions
predictions = tf.argmax(logits, axis=1)

# Calculate accuracy
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(y_test, predictions)
acc_value = accuracy.result().numpy()


print("Accuracy", acc_value)

Accuracy 0.64325
